<a href="https://colab.research.google.com/github/rsemihkoca/YouDo-ds-bc/blob/main/You_Do_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',names=['user_id', 'item_id', 'rating', 'timestamp'])
r = df.pivot(index='user_id', columns='item_id', values='rating').values



/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


# Part 0 Train-Test Split

In [4]:
irow, jcol = np.where(~np.isnan(r))
n=int(len(irow)*0.2)

idx = np.random.choice(np.arange(len(irow)), n, replace=False)

In [5]:

test_irow = irow[idx]
test_jcol = jcol[idx]
jcol[idx].shape

(20000,)

In [6]:
r_original=r.copy()
r_copy = r.copy()
l=0
for i in range(n):
  r_copy[test_irow[i]][test_jcol[i]] = np.nan
  l+=1

In [7]:
np.isnan(r_copy).sum()-np.isnan(r_original).sum()

20000

In [8]:
users,items=r_copy.shape

In [52]:
def train_loss(r,b_users,b_items,irow, jcol):
  Train_Loss=0
  for u, j in zip(irow, jcol):
    Train_Loss+=np.nansum((r[u][j] - (b_users[u] + b_items[j]))**2)*0.5
  return Train_Loss

def test_loss(r,b_users,b_items,test_irow, test_jcol):
  Test_Loss=0
  for u, j in zip(test_irow, test_jcol):
    Test_Loss+=np.nansum((r[u][j] -  (b_users[u] + b_items[j]))**2)*0.5
  return Test_Loss



# Part 1 Optimization 

In [53]:
b_users=np.random.rand(users,1)
grad_b_users=np.random.rand(users,1)

b_items=np.random.rand(items,1)
grad_b_items=np.random.rand(items,1)

alpha=(3)*(10**-2)
max_iter=100
r_pred=np.full((r_copy.shape),np.nan)

for iter in range(max_iter):
  for i in range(users):
    for j in range(items):
      if np.isnan(r_copy[i][j]): # zaten bilmediğim değeri tahmin edemem NANları nasıl tahmin ederim
            continue
      else:
        try:

          r_pred[i][j]= b_users[i] + b_items[j]

          grad_b_users[i]=-(r_copy[i][j]-r_pred[i][j])
          grad_b_items[j]=-(r_copy[i][j]-r_pred[i][j])

          b_users_prev=b_users.copy()

          b_users[i] = b_users[i]- alpha * grad_b_users[i]
          b_items[j] = b_items[j]- alpha * grad_b_items[j]


        except ValueError:
          print("ValueError encountered")
  Train_Loss= train_loss(r_copy,b_users,b_items,irow, jcol)
  Test_Loss= test_loss(r_original,b_users,b_items,test_irow, test_jcol)

  if(iter%1 == 0):
    print(f"iteration: ({iter}) , gradient: {np.linalg.norm(grad_b_users)+np.linalg.norm(grad_b_items):.3f}, Train_Loss: {Train_Loss:.3f}, Test_Loss: {Test_Loss:.3f}")
            
  # if np.linalg.norm(b_users-b_users_prev) < 0.000001:
  #   print(f"I do early stoping at iteration {iter}")
  #   break


iteration: (0) , gradient: 97.509, Train_Loss: 84274.002, Test_Loss: 21298.478
iteration: (1) , gradient: 81.732, Train_Loss: 38834.200, Test_Loss: 10061.503
iteration: (2) , gradient: 76.219, Train_Loss: 36302.066, Test_Loss: 9441.357


KeyboardInterrupt: ignored

In [54]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

a=pd.DataFrame(r_pred[:5,:10]).fillna('-', inplace=False)
b=pd.DataFrame(r_copy[:5,:10]).fillna('-', inplace=False)
print("                  r_pred vs r_copy\n")
pd.concat([a,b],axis=0)

                  r_pred vs r_copy



,0,1,2,3,4,5,6,7,8,9
0,3.75,3.08,2.84,3.29,3.23,3.21,3.80,3.77,3.89,-
1,-,-,-,-,-,-,-,-,-,3.27
2,-,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-,-
4,3.29,2.57,-,-,-,-,-,-,-,-
0,5.00,3.00,4.00,3.00,3.00,5.00,4.00,1.00,5.00,-
1,-,-,-,-,-,-,-,-,-,2.00
2,-,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-,-
4,4.00,3.00,-,-,-,-,-,-,-,-


# Part 2 Regularized Model

In [24]:
def train_loss_regularized(r,b_users,b_items,irow, jcol,lamb):
  Train_Loss=0
  for u, j in zip(irow, jcol):
    Train_Loss+=np.nansum((r[u][j] - (b_users[u] + b_items[j]))**2)*0.5 + (b_users[u]**2 + b_items[j]**2)*lamb*0.5
  return float(Train_Loss[0])

def test_loss_regularized(r,b_users,b_items,test_irow, test_jcol,lamb):
  Test_Loss=0
  for u, j in zip(test_irow, test_jcol):
    Test_Loss+=np.nansum((r[u][j] -  (b_users[u] + b_items[j]))**2)*0.5 + (b_users[u]**2 + b_items[j]**2)*lamb*0.5
  return float(Test_Loss[0])



In [80]:
import numpy as np




def Regularized_SGD(lamb):

  b_users=np.random.rand(users,1)
  grad_b_users=np.random.rand(users,1)

  b_items=np.random.rand(items,1)
  grad_b_items=np.random.rand(items,1)

  r_pred=np.full((r_copy.shape),np.nan)

  alpha=(3)*(10**-2)
  Test_Loss_list=[]
  Info=False
  max_iter=3

  for iter in range(max_iter):
    for i in range(users):
      for j in range(items):
        if np.isnan(r_copy[i][j]): # zaten bilmediğim değeri tahmin edemem NANları nasıl tahmin ederim
              continue
        else:
          try:

            r_pred[i][j]= b_users[i] + b_items[j]

            grad_b_users[i]=-(r_copy[i][j]-r_pred[i][j]) + lamb * b_users[i]
            grad_b_items[j]=-(r_copy[i][j]-r_pred[i][j]) + lamb * b_items[j]

            b_users_prev=b_users.copy()

            b_users[i] = b_users[i]- alpha * grad_b_users[i]
            b_items[j] = b_items[j]- alpha * grad_b_items[j]


          except ValueError:
            print("ValueError encountered")
    Train_Loss= train_loss_regularized(r_copy,b_users,b_items,irow, jcol,lamb)
    Test_Loss= test_loss_regularized(r_original,b_users,b_items,test_irow, test_jcol,lamb)
    Test_Loss_list.append(Test_Loss)
    if Info:
      if(iter%1 == 0):
        print(f"iteration: ({iter}) , gradient: {(np.linalg.norm(grad_b_users)+np.linalg.norm(grad_b_items)):.3f}, Reg_Train_Loss: {Train_Loss:.3f}, Reg_Test_Loss: {Test_Loss:.3f}")
        
  return -min(Test_Loss_list)

max_iter=3
lamb=0.001

Regularized_SGD(lamb)


    # return r_pred,b_users, b_items

KeyboardInterrupt: ignored

# Part 2 HyperParam Opt

In [59]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=0314cf812c5489f3dbfba1b29c34f575d30512fb6d30dd8e8cf5f0cc041acfee
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [97]:

#Regularized_SGD(max_iter,lamb,Info=False)

from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'lamb': (0.00005,0.1)}

optimizer = BayesianOptimization(
    f=Regularized_SGD,
    pbounds=pbounds,
    random_state=1,
)

In [98]:
optimizer.maximize(
    init_points=0.0001,
    n_iter=10,
)

|   iter    |  target   |   lamb    |
-------------------------------------
|  1        | -1.21e+04 |  0.04173  |
|  2        | -1.56e+04 |  0.1      |
|  3        | -1.209e+0 |  0.04174  |
|  4        | -1.268e+0 |  0.05158  |
|  5        | -9.464e+0 |  5e-05    |
|  6        | -9.818e+0 |  0.005787 |
|  7        | -9.818e+0 |  0.005787 |
|  8        | -9.818e+0 |  0.005787 |
|  9        | -9.818e+0 |  0.005787 |
|  10       | -9.818e+0 |  0.005787 |
|  11       | -9.818e+0 |  0.005787 |


In [116]:
print(f"Best lambda value: {optimizer.max['params']['lamb']:.5f}\nRegular_Test_Loss: {-optimizer.max['target']:.5f}")


Best lambda value: 0.00005
Regular_Test_Loss: 9464.40593
